In [ ]:
%run '/content/drive/MyDrive/Colab Notebooks/Hamoye Intership/classification/log_regression.ipynb'

In [15]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(log_reg, normalized_train, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.52768566, 0.53051643, 0.5       , 0.56892231, 0.51826576])